## Signal Analysis
This script is aimed to analyze the signals proposed for BCI usage for the MACI proposal.

In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.signal as sgn
import scipy.stats as sta

from Signal import Signal
from Dataset import Dataset

In [ ]:
PATH = './stim_23'
PATH_RAW = os.path.join(PATH, 'train_data')
PATH_FVS = os.path.join(PATH, 'fv_data')
PATH_RESULT = os.path.join(PATH, 'results')
if not os.path.exists(PATH_FVS):
    os.mkdir(PATH_FVS)

SF = 200

ORDER = 4
BP_LO = 4
BP_HI = 50
NOTCH = 50

WINDOW = 512
STRIDE = 1

FREQ = 23 #freq none takes it from the file name
BW = 1
HARMS = [1, 2]
APPLY_SNR = True

LAB_RELS = {
    99: 0,
    1: 1,
    2: 2
}

SOURCE = 'freq'

CHANNELS = None
SUBJECTS = None
SESSIONS = None
LABELS = None

## Matching
The idea is to match the features from stimulation to others both in no stimulation and noise

In [ ]:
sig = Signal('test', 1, '2020-20-20')
sig.load_raw(os.path.join(PATH_RAW, 'HC 4 23Hz 2021-04-21.txt'), LAB_RELS)
sig.process(SF, ORDER, BP_LO, BP_HI, NOTCH)
sig.make_fvs(SF, WINDOW, STRIDE, FREQ, BW, HARMS, APPLY_SNR)

labels = sig.get_Y()

In [ ]:
def match_pattern(X, y, thresh = 0.8):
    matches = {}
    index = np.arange(X.shape[0])
    label_mask = y[:, 1] == 1.0
    for j, feat in zip(index[label_mask], X[label_mask]):
        window_mask = ~label_mask
        window_mask[j:j+512] = False
        corrs = np.apply_along_axis(lambda x: sta.spearmanr(x, feat)[0], 1, X[window_mask])
        for i, corr in zip(index[window_mask], corrs):
            if abs(corr) >= thresh:
                corr_dist = corr * abs(i-j) / X.shape[0]
                try:
                    if matches[i] < corr_dist:
                        matches[i] = round(abs(corr_dist), 2)
                except KeyError:
                    matches[i] = corr_dist         

    return list(matches.items())

In [ ]:
times = sig.get_time_X(CHANNELS, LABELS)
dataframe = pd.DataFrame(columns=["ch", "pos", "corr", "count", "to_reduce"])
for ch in sig.get_chans():
    matches = match_pattern(times[ch][2000:6000], labels[2000:6000])
    matches_data = []
    if len(matches) > 0:
        join_value = matches[0][0]
        matches_data.append((ch, matches[0][0], matches[0][1], 1, f'{join_value}{ch}'))
        for i in range(1, len(matches)):
            if abs(matches[i][0] - matches[i-1][0]) > 32: #joins on reduce if two highly correlated value are 8 samples appart
                join_value = matches[i][0]
            matches_data.append((ch, matches[i][0], matches[i][1], 1, f'{join_value}{ch}'))
    dataframe = pd.concat((dataframe, pd.DataFrame(matches_data, columns = dataframe.columns)))

fig, axs = plt.subplots(1, 1)
fig.set_figheight(5)
fig.set_figwidth(10)
axs.set_xlim((1500, 4000))
axs.set_xlabel("signal samples", fontsize=12)
axs.set_ylabel("ch", fontsize=12)
suptitle = fig.suptitle(f'Matches for S2', fontsize=15)
#grouping values with similar positions, by moving position back one order of magnitude (/100) and grouping rows with same pos
dataframe_reduced = dataframe.groupby(dataframe["to_reduce"]).aggregate({'corr': 'max', 'ch': 'first', 'pos': 'first', 'count': 'sum'})
sns.scatterplot(dataframe_reduced, x="pos", y="ch", hue="corr", size="count", sizes=(50, 250), ax=axs)

plt.savefig(os.path.join(PATH_RESULT, suptitle.get_text()))